# Explore and cluster the neighborhoods of Toronto
#### lets get the data from last time and only use those boroughs which have 'toronto' in their name

In [1]:
# read the postal codes with locations we prepared in the previous assignment
import pandas as pd
df_TN_1 = pd.read_csv('PC_TorontoGEo.csv')
df_TN_1.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [2]:
# We will only use the Boroughs which contain 'Toronto'
df_TN_2 = df_TN_1[df_TN_1['Borough'].str.contains('Toronto')]
df_TN_2.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [3]:
# Lets check how many we've got
df_TN_2.shape

(39, 5)

## Lets show where the Neighborhoods in Toronto are located
### Ill Take Union Station as center (in blue) and the Neighborhoods (in red)

In [4]:
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library    

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    brotlipy-0.7.0             |py36h8c4c3a4_1000         346 KB  conda-forge
    chardet-3.0.4              |py36h9f0ad1d_1006         188 KB  conda-forge
    cryptography-2.9.2         |   py36h45558ae_0         613 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    pandas-1.0.3               |   py36h830a2c2_1        11.1 MB  conda-

### We have to replace the apostrophe's in the Neighborhoods if we want to use them as labels in the maps!

In [5]:
# lets replace the aphostrophe with nothing 
df_TN_3 = df_TN_2.replace(to_replace="\'",value="",regex=True )
df_TN_3.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queens Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [6]:
# lets take Union Station as the center
latitude = 43.6482047
longitude = -79.3715274
Toronto_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Toronto central

# add a red circle marker to represent the City Center
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='blue',
    popup='Union Station',
    fill = True,
    fill_color = 'blue',
    fill_opacity = 0.6
).add_to(Toronto_map)

# add the Boroughs as blue circle markers
for lat, lng, label in zip(df_TN_3.Latitude, df_TN_3.Longitude, df_TN_3.Neighborhood):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(Toronto_map)

# Show the map
Toronto_map